# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Import the WeatherPy_database.csv file. 
file_path = "../Vacation_Search/WeatherPy_vacation.csv"

# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv(file_path)

# Display sample data
vacation_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,81.36,overcast clouds,-0.6000,73.0833,Pebbles Inn
1,Atuona,PF,77.81,few clouds,-9.8000,-139.0333,Pearl Resort
2,Saint-Philippe,RE,80.19,scattered clouds,-21.3585,55.7679,Le Baril
3,Port Elizabeth,ZA,71.31,clear sky,-33.9180,25.5701,Waterford Hotel
4,Bubaque,GW,80.04,overcast clouds,11.2833,-15.8333,Chez Julio
...,...,...,...,...,...,...,...
141,Kerema,PG,85.37,scattered clouds,-7.9631,145.7785,Kerema Hotel
142,Acapulco,MX,76.82,smoke,16.8634,-99.8901,Hotel del Valle
143,San Juan Del Sur,NI,77.29,overcast clouds,11.2529,-85.8705,Hotel Colonial
144,Eyl,SO,79.11,clear sky,7.9803,49.8164,فندق ايل دابي


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [4]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [5]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City'] =='Cairns']
vacation_end = vacation_df.loc[vacation_df['City'] =='Cairns']
vacation_stop1 = vacation_df.loc[vacation_df['City'] =='Bowen']
vacation_stop2 = vacation_df.loc[vacation_df['City'] =='Mackay']
vacation_stop3 = vacation_df.loc[vacation_df['City'] =='Yeppoon']

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [6]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], axis=0)

# Display sample data
itinerary_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
96,Cairns,AU,84.15,light rain,-16.9167,145.7667,Cairns Queens Court Accommodation
95,Bowen,AU,79.50,overcast clouds,-20.0167,148.2333,Whitsunday Sands Resort
87,Mackay,AU,75.60,overcast clouds,-21.1500,149.2000,Mackay Oceanside Central Apartment Hotel
47,Yeppoon,AU,73.18,overcast clouds,-23.1333,150.7333,Blossom's on Seaspray
96,Cairns,AU,84.15,light rain,-16.9167,145.7667,Cairns Queens Court Accommodation


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [7]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["City", "Lng", "Lat"]].copy()

# Display sample data
waypoints_df

,City,Lng,Lat
96,Cairns,145.7667,-16.9167
95,Bowen,148.2333,-20.0167
87,Mackay,149.2000,-21.1500
47,Yeppoon,150.7333,-23.1333
96,Cairns,145.7667,-16.9167


## Use GeoViews to create map that shows the four cities in the itinerary

In [8]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = 0.001,
    scale = 500,
    color = "City"
)

In [9]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [10]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [11]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'b929e1e6e6ee40789edf1c20677e75f9',
 'waypoints': '-16.9167,145.7667|-20.0167,148.2333|-21.15,149.2|-23.1333,150.7333|-16.9167,145.7667'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [12]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = requests.get(base_url, params=params).json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [145.7667, -16.9167], 'original_index': 0},
     {'location': [148.2333, -20.0167], 'original_index': 1},
     {'location': [149.2, -21.15], 'original_index': 2},
     {'location': [150.7333, -23.1333], 'original_index': 3},
     {'location': [145.7667, -16.9167], 'original_index': 4}],
    'units': 'metric',
    'distance': 2196445,
    'distance_units': 'meters',
    'time': 85689.423,
    'legs': [{'distance': 550111,
      'time': 21691.06,
      'steps': [{'from_index': 0,
        'to_index': 4,
        'distance': 105,
        'time': 7.611,
        'instruction': {'text': 'Drive southeast on McLeod Street.'}},
       {'from_index': 4,
        'to_index': 8,
        'distance': 143,
        'time': 12.117,
        'instruction': {'text': 'Turn left onto Upward Street.'}},
       {'from_index': 8,
        'to_index': 61,
        'distance': 2332,
        'time': 147.909,
        'in

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [15]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [16]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [17]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head(10)

,longitude,latitude
0,145.766302,-16.916987
1,145.766407,-16.917120
2,145.766559,-16.917256
3,145.766859,-16.917573
4,145.766964,-16.917685
5,145.767787,-16.917055
6,145.767831,-16.917021
7,145.767912,-16.916962
8,145.768017,-16.916881
9,145.768074,-16.916951


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [18]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "red")

In [19]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)
   .Path.I   :Path   [longitude,latitude]